In [1]:
import json
import os
import nltk
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from datasets import Dataset

C:\Users\48798\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

In [3]:
df = pd.read_csv('eval_df.csv')

In [4]:
eval_df = pd.DataFrame()
eval_df['text_tokens'] = df['text'].apply(preprocess_text)
eval_df['summary_tokens'] = df['summary'].apply(preprocess_text)

In [5]:
from functions_utils import rank_sentences, Feng
feng_instance_loaded = Feng(preprocessed_df=eval_df)
feng_instance_loaded.word2vec_model = Word2Vec.load("word2vec.model")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\48798\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\48798\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\48798\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
df.rename({'tekst' : 'bart_summary'}, axis = 1, inplace = True)

In [7]:
from nltk.tokenize import sent_tokenize
from datetime import datetime

start_time = datetime.now()
df['w2v_summary'] = df['text'].apply(lambda x: rank_sentences(x, feng_instance_loaded)[0])
end_time = datetime.now()
execution_time = (end_time - start_time).total_seconds()
print(f"Czas predykcji (500 przykładów): {execution_time:.6f} sekund")

Czas predykcji (500 przykładów): 3.134546 sekund


In [8]:
df['bart_summary'] = df['bart_summary'].apply(lambda x: str(x)[19:])

In [9]:
df = df.iloc[:, 1:]
df.to_csv('ready_to_compare.csv')